In [9]:
from helper import *
import textract
import joblib
import os
from time import sleep
from timeit import default_timer
import asyncio
from pyppeteer import launch
from pyppeteer import errors
import nest_asyncio
nest_asyncio.apply()
from multiprocessing import Pool

In [ ]:
async def main():
    
    count, step = 0, 5
    range_start, range_stop = 100_050, 101_000
    
    files = os.listdir("/home/lukas/Downloads/")
    prev_indices = [int(file[-10:-4]) for file in files]
    
    start = default_timer()
    
    for start_idx in range(range_start, range_stop, step):
        
        urls = [get_url(idx) for idx in range(start_idx, start_idx+step)
               if idx not in prev_indices]
        
        if not urls:
            continue
        
        browser = await launch(headless=False)

        for url in urls:    
            page = await browser.newPage()
            page.setDefaultNavigationTimeout(5000)
            try:
                await page.goto(url)
            except errors.TimeoutError:
                await browser.close()
                browser = await launch(headless=False)
                page = await browser.newPage()
                page.setDefaultNavigationTimeout(5000)
                try:
                    await page.goto(url)
                except errors.TimeoutError:
                    print(f"Could not open {url}")
                
            button = await page.J("[aria-label='Download summary file']")
            try:
                await button.click()
                count += 1
                print(f'Downloaded {url} (file {count})')
            except AttributeError:
                pass

        await page.waitFor(2000)
        await browser.close()
        
        el = default_timer() - start
        print(f'Scanned {start_idx}-{start_idx+step} after {el/60:.1f} min')

asyncio.get_event_loop().run_until_complete(main())

Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100118 (file 1)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100119 (file 2)
Scanned 100115-100120 after 0.1 min
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100120 (file 3)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100121 (file 4)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100122 (file 5)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100123 (file 6)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100124 (file 7)
Scanned 100120-100125 after 0.3 min
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100125 (file 8)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100126 (file 9)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100127 (file 10)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100128 (file 11)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/100129 (file 12)
Scanned 10012

In [13]:
def main(range_start, range_stop):

    count, step = 0, 5

    files = os.listdir("/home/lukas/Downloads/")
    prev_indices = [int(file[-10:-4]) for file in files]

    start = default_timer()

    for start_idx in range(range_start, range_stop, step):

        urls = [get_url(idx) for idx in range(start_idx, start_idx+step)
               if idx not in prev_indices]

        if not urls:
            continue

        driver = webdriver.Chrome(options=get_chrome_options(headless=True))

        for url in urls:
            
            driver.get(url)
            
            for element in driver.find_elements(By.CLASS_NAME, 
                                                'MuiButtonBase-root'):
                
                attribure = element.get_attribute('aria-label')
                
                if attribure == 'Download summary file':
                    element.click()
                    count += 1
                    print(f'Downloaded {url} (file {count})')
                    
        el = default_timer() - start
        print(f'Scanned {start_idx}-{start_idx+step} after {el/60:.1f} min')

In [14]:
with Pool(2) as pool:
    params = [(101_000, 101_500), (101_500, 102_000)]
    pool.starmap(main, params)
pool.close()

Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101528 (file 1)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101110 (file 1)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101529 (file 2)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101111 (file 2)
Scanned 101525-101530 after 0.1 min
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101112 (file 3)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101113 (file 4)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101114 (file 5)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101530 (file 3)
Scanned 101110-101115 after 0.1 min
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101531 (file 4)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101532 (file 5)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101533 (file 6)
Downloaded https://rnaloops.cs.put.poznan.pl/search/details/101115 (file 6)
Downloaded https

KeyboardInterrupt: 

In [ ]:
def get_content():

    file = [x for x in os.listdir() if str(last_idx) in x]
    file = file[0]

    try:
        content = (textract.process(file).decode('utf-8').split("\n"))
        os.remove(file)
    except UnicodeDecodeError:
        continue

    result = get_data_dict(content)
    joblib.dump(result, str(last_idx))
    